In [1]:
import helpers
import pandas as pd
import time
import copy
from helpers import musicbrainz,langchainhelpers
from tqdm import tqdm
from langchain.prompts import ChatPromptTemplate
pd.set_option("display.max_colwidth",1000)

In [ ]:
output_parser=langchainhelpers.customOutputFunctionsParser()

In [ ]:
#load functions of music brainz
all_available_functions = {"searchdetailsrefined":musicbrainz.searchdetailsrefined,
             "getalbumdetails":musicbrainz.getalbumdetails,
             "getartistdetails":musicbrainz.getartistdetails,
             "gettrackdetailsrefined":musicbrainz.gettrackdetailsrefined}


In [ ]:
prompt1 = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an Music Aficionado and know a lot about this field."),
            ("human", "You also have access to a music database, so make calls to the relevant functions if necessary: {input}")
        ]
    )

In [ ]:
results=helpers.download_file("data/Musicdata/intermediate_results.pkl")

In [ ]:
results

# Experimentation

# Only one function available

In [2]:
hard_questions_df=pd.read_csv("data/Musicdata/Hard_questions.csv")
hard_questions_df

,Question,Correct Answer
0,Its the year 2023. What is the age of the Person whom Elton John sang Cold Heart with?,28
1,Give me the 6th song of the album in which Bruno Mars released Moonshine?,When I Was Your Man
2,Who is the common artist of Tere Naina from My Name Is Khan and Tu Hi Mera from Jannat 2?,Shafqat Amanat Ali
3,Name one artist whose names start with A who are part of the band that released Shakermaker in the Definitely Maybe.,Alan White/Andy Bell
4,What is the birth place of the artist who released 24K Magic?,Honolulu
5,Name the artist who released Raising Sand and I Know Who Holds Tomorrow and was born in Champaign.,Alison Krauss
6,Who is the producer of the 2nd song of the album Fearless by Taylor Swift?,Nathan Chapman
7,What is the capital of the country that the artist who released Raising Sand was born in. The artist must not be from a country in North America?,London
8,"In the year that Taylor Swift released Lover and Cold Play released Everyday Life, England won the 50 over cricket world cup for the first time. What year was it?",2019
9,"Out of these four ÷ - + * , which is one the symbol that Ed Sheeran's music albums are not named after?",*


In [ ]:
available_functions={k:v for k,v in all_available_functions.items() if  v.__name__=="searchdetailsrefined"}
available_functions

In [ ]:
results=[]

In [ ]:
for question in tqdm(hard_questions_df["Question"]): 
    prompt=copy.deepcopy(prompt1)
    results.append(langchainhelpers.run_llm_chain_more_than_once(prompt,available_functions,question,1000,1,5))
    helpers.upload_file("data/Musicdata/intermediate_results.pkl",results)

In [ ]:
results

In [ ]:
helpers.upload_file("data/Musicdata/hard_questions_with_one_function_available_prompt_1.pkl",results)

In [4]:
results=helpers.download_file("data/Musicdata/hard_questions_with_one_function_available_prompt_1.pkl")

In [5]:

results=pd.DataFrame(results)
results=pd.merge(left=results,right=hard_questions_df)
results

,Question,steps,answer,Correct Answer
0,Its the year 2023. What is the age of the Person whom Elton John sang Cold Heart with?,"[{'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Elton John', 'entity_type': 'artist'}}, {'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Cold Heart', 'entity_type': 'track', 'artist_name': 'Elton John'}}]","The person whom Elton John sang ""Cold Heart"" with is Dua Lipa.",28
1,Give me the 6th song of the album in which Bruno Mars released Moonshine?,"[{'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Bruno Mars', 'entity_type': 'artist', 'album_name': 'Moonshine'}}, {'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Bruno Mars', 'entity_type': 'album', 'album_name': 'Moonshine'}}]","I'm sorry, but I couldn't find any information about an album called ""Moonshine"" released by Bruno Mars in the music database. Please make sure the album name is correct and try again.",When I Was Your Man
2,Who is the common artist of Tere Naina from My Name Is Khan and Tu Hi Mera from Jannat 2?,"[{'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Tere Naina', 'entity_type': 'track', 'album_name': 'My Name Is Khan'}}]","The common artist of ""Tere Naina"" from the album ""My Name Is Khan"" and ""Tu Hi Mera"" from the album ""Jannat 2"" is Shafqat Amanat Ali.",Shafqat Amanat Ali
3,Name one artist whose names start with A who are part of the band that released Shakermaker in the Definitely Maybe.,"[{'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Definitely Maybe', 'entity_type': 'album'}}]","The band that released ""Shakermaker"" in the album ""Definitely Maybe"" is Oasis.",Alan White/Andy Bell
4,What is the birth place of the artist who released 24K Magic?,"[{'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': '24K Magic', 'entity_type': 'album'}}]","The artist who released ""24K Magic"" is Bruno Mars. The birthplace of Bruno Mars is Honolulu, Hawaii, United States.",Honolulu
5,Name the artist who released Raising Sand and I Know Who Holds Tomorrow and was born in Champaign.,"[{'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Raising Sand', 'entity_type': 'album'}}, {'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'I Know Who Holds Tomorrow', 'entity_type': 'album'}}]","The artist who released both ""Raising Sand"" and ""I Know Who Holds Tomorrow"" and was born in Champaign is Alison Krauss.",Alison Krauss
6,Who is the producer of the 2nd song of the album Fearless by Taylor Swift?,"[{'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Fearless', 'entity_type': 'album', 'artist_name': 'Taylor Swift'}}]",The producer of the 2nd song of the album Fearless by Taylor Swift is Taylor Swift herself.,Nathan Chapman
7,What is the capital of the country that the artist who released Raising Sand was born in. The artist must not be from a country in North America?,"[{'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Raising Sand', 'entity_type': 'album'}}, {'Function_name': 'searchdetailsrefined', 'Function_arguments': {'query': 'Robert Plant', 'entity_type': 'artist', 'artist_name': 'Robert Plant'}}]","The artist Robert Plant, who released the album ""Raising Sand,"" was born in West Bromwich, United Kingdom. The capital of the United Kingdom is London.",London
8,"In the year that Taylor Swift released Lover and Cold Play released Everyday Life, England won the 50 over cricket world cup for the first time. What year was it?",[],"In the year that Taylor Swift released ""Lover"" and Coldplay released ""Everyday Life"", England won the 50 over cricket world cup for the first time. The cricket world cup victory for England happened in the year 2019.",2019
9,"Out of these four ÷ - + * , which is one the symbol that Ed Sheeran's music albums are not named after?","[{'Fun

# 3 subfunctions available

In [ ]:
hard_questions_df=pd.read_csv("data/Musicdata/Hard_questions.csv")
hard_questions_df

In [ ]:
available_functions={k:v for k,v in all_available_functions.items() if  v.__name__!="searchdetailsrefined"}
available_functions

In [ ]:
results=[]

In [ ]:
for question in tqdm(hard_questions_df["Question"]): 
    prompt=copy.deepcopy(prompt1)
    results.append(langchainhelpers.run_llm_chain_more_than_once(prompt,available_functions,question,1000,1,5))
    helpers.upload_file("data/Musicdata/intermediate_results.pkl",results)

In [ ]:
helpers.upload_file("data/Musicdata/hard_questions_with_6_subfunctions_available_prompt_1.pkl",results)

In [6]:
results=helpers.download_file("data/Musicdata/hard_questions_with_6_subfunctions_available_prompt_1.pkl")

In [ ]:

results=pd.DataFrame(results)
results=pd.merge(right=results,left=hard_questions_df)
results